In [1]:
import json
import pandas as pd
import folium
from geopy.distance import geodesic

### Para obtener las distancias se ha realizado una consulta a la api de [OpenStreetMap](https://overpass-turbo.eu/)

<div style="text-align: center;">
  <a href="https://www.openstreetmap.org/#map=7/53.462/-8.240">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Openstreetmap_logo.svg/1024px-Openstreetmap_logo.svg.png" alt="OpenStreetMap" width="200" height="200">
  </a>
  <a href="https://overpass-turbo.eu/">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Overpass-turbo.svg/1024px-Overpass-turbo.svg.png" alt="Overpass Turbo" width="200" height="200">
  </a>
</div>


In [2]:
house = pd.read_csv('/TFM Mercado Inmobiliario - Kschool/Metodología de Modelado/9. Código y datasets/Barcelona/Datasets/result_barcelona_districs_zip_final.csv', sep = ";")

In [3]:
gimnasio = pd.read_csv('/TFM Mercado Inmobiliario - Kschool/Metodología de Modelado/9. Código y datasets/Barcelona/Datasets/Distancia_servicios/Dataset_coordenadas_servicios/gimnasios.csv')

In [4]:
nom = gimnasio.columns[0]
nom_dist = nom + '_distancia'
nom_dist

'Gimnasio _distancia'

In [ ]:
def obtener_estacion_mas_cercana(house_row, cinema_df):
    if 'latitude' not in house_row or 'longitude' not in house_row:
        raise ValueError("La fila de la casa no tiene columnas 'latitude' o 'longitude'")
    house_coords = (house_row['latitude'], house_row['longitude'])
    cinema_df[nom_dist] = cinema_df.apply(lambda cinema_row: 
                                              geodesic(house_coords, (cinema_row['latitude'], cinema_row['longitude'])).meters, 
                                              axis=1)

    closest_station_row = cinema_df.loc[cinema_df[nom_dist].idxmin()]

    estacion_cercana = closest_station_row[nom]
    distancia_minima = closest_station_row[nom_dist]

    return estacion_cercana, distancia_minima

gimnasio[['longitude', 'latitude']] = gimnasio[['longitude', 'latitude']].apply(pd.to_numeric, errors='coerce')

print(gimnasio.columns)

house[['latitude', 'longitude']] = house[['latitude', 'longitude']].apply(pd.to_numeric, errors='coerce')

house[[nom, nom_dist]] = house.apply(
    lambda row: pd.Series(obtener_estacion_mas_cercana(row, gimnasio)), axis=1
)

Index(['Gimnasio ', 'longitude', 'latitude'], dtype='object')
      propertyCode                                          thumbnail  \
0         99117414  https://img4.idealista.com/blur/WEB_LISTING-M/...   
1        101300670  https://img4.idealista.com/blur/WEB_LISTING-M/...   
2        104772587  https://img4.idealista.com/blur/WEB_LISTING-M/...   
3        100770574  https://img4.idealista.com/blur/WEB_LISTING-M/...   
4        105932101  https://img4.idealista.com/blur/WEB_LISTING-M/...   
...            ...                                                ...   
8913     105882581  https://img4.idealista.com/blur/WEB_LISTING-M/...   
8914      81411842  https://img4.idealista.com/blur/WEB_LISTING-M/...   
8915     103461663  https://img4.idealista.com/blur/WEB_LISTING-M/...   
8916      98964029  https://img4.idealista.com/blur/WEB_LISTING-M/...   
8917     105093241  https://img4.idealista.com/blur/WEB_LISTING-M/...   

     externalReference  numPhotos    floor       price proper

In [7]:
house.to_csv("/TFM Mercado Inmobiliario - Kschool/Metodología de Modelado/9. Código y datasets/Barcelona/Datasets/Distancia_servicios/Dataset_distancias/" + nom_dist + ".csv", index=False, sep=";")
